<a href="https://colab.research.google.com/github/fennel-s/colab/blob/main/StableDiffusionUI(NovelAILeaks)_%E4%BA%91%E7%9B%98%E5%AE%89%E8%A3%85%E7%89%88release.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# StableDiffusion NovelAi Google云盘安装版

账号首次执行：依次点击除第4步以外所有播放键

第二次及以后：点击1、2、5

Credits:

https://github.com/AUTOMATIC1111/stable-diffusion-webui

https://github.com/TheLastBen/fast-stable-diffusion


In [ ]:
#@title 1、授权挂载Google云盘
from google.colab import drive

#@markdown 配置stable-diffusion-webui所在文件夹名称
SD_Folder_Name='sd' #@param {type:"string"}

!nvidia-smi --query-gpu=name,memory.total,memory.free --format=csv

drive.mount('/content/gdrive')
!mkdir -p /content/gdrive/MyDrive/$SD_Folder_Name
sd_dir="/content/gdrive/MyDrive/"+SD_Folder_Name
webui_dir="/content/gdrive/MyDrive/"+SD_Folder_Name+"/stable-diffusion-webui"

In [ ]:
#@title 2、初始化临时运行环境【每次新开会话时执行一次】
import os,sys,time
from IPython.utils import capture
from IPython.display import clear_output
from IPython.display import HTML
from subprocess import getoutput

#安装环境依赖

#@markdown 启用快速安装（程序启动有问题时尝试重开会话并取消勾选）
Fast_Install_Dependencies=True #@param {type:"boolean"}

%cd /content/
if Fast_Install_Dependencies and '3.7' in sys.version:
  with capture.capture_output() as cap:
    !wget -O Dependencies_AUT.7z.001 https://github.com/TheLastBen/fast-stable-diffusion/raw/main/Dependencies/Dependencies_AUT.1
    !wget -O Dependencies_AUT.7z.002 https://github.com/TheLastBen/fast-stable-diffusion/raw/main/Dependencies/Dependencies_AUT.2
    !7z x Dependencies_AUT.7z.001
    !cp -r /content/usr/local/lib/python3.7/dist-packages /usr/local/lib/python3.7/
    !rm -r /content/usr
    !rm Dependencies_AUT.7z.001 Dependencies_AUT.7z.002
else:
  # Install triton
  !pip install -q -U --pre triton

if not os.path.exists('/tools/node/bin/lt'):
  !npm install -g localtunnel

gpuinfo = getoutput('nvidia-smi --query-gpu=name --format=csv,noheader')
!pip install -q tensorflow tensorflow-io
!pip install -q git+https://github.com/KichangKim/DeepDanbooru.git@edf73df4cdaeea2cf00e9ac08bd8a9026b7a7b26#egg=deepdanbooru
!pip install -q scikit-image --upgrade

# Install xformers for python version < 3.10
if int(sys.version.split('.')[1])<10:
  !echo "$gpuinfo"
  if 'T4' in gpuinfo:
    if '3.7' in sys.version:
      %pip install -q https://github.com/daswer123/xformers_prebuild_wheels/raw/main/Google%20Collab/T4/python37/xformers-0.0.14.dev0-cp37-cp37m-linux_x86_64.whl
    elif '3.8' in sys.version:
      %pip install -q https://github.com/daswer123/xformers_prebuild_wheels/raw/main/Google%20Collab/T4/python38/xformers-0.0.14.dev0-cp38-cp38-linux_x86_64.whl
    elif '3.9' in sys.version:
      %pip install -q https://github.com/daswer123/xformers_prebuild_wheels/raw/main/Google%20Collab/T4/python39/xformers-0.0.14.dev0-cp39-cp39-linux_x86_64.whl
    else:
      !echo "install xformers failed: unsupported python version."
  elif 'P100' in gpuinfo:
    %pip install -q https://github.com/TheLastBen/fast-stable-diffusion/raw/main/precompiled/P100/xformers-0.0.13.dev0-py3-none-any.whl
  elif 'V100' in gpuinfo:
    %pip install -q https://github.com/TheLastBen/fast-stable-diffusion/raw/main/precompiled/V100/xformers-0.0.13.dev0-py3-none-any.whl
  elif 'A100' in gpuinfo:
    %pip install -q https://github.com/TheLastBen/fast-stable-diffusion/raw/main/precompiled/A100/xformers-0.0.13.dev0-py3-none-any.whl
  else:
    print('it seems that your GPU is not supported at the moment, install xformers failed.')

!echo done

In [ ]:
#@title /3、下载webui程序和模型文件到云盘（仅首次安装到云盘时执行！）
import os,sys,time
from IPython.utils import capture
from IPython.display import clear_output

#@markdown 先在云盘里手动清空回收站

#@markdown 勾选以下载latest版模型（7GB，请确保云盘有足够的存储空间。）：
Use_Latest_Model=False #@param {type:"boolean"}
#@markdown 勾选以下载NovelAi hypernetwork文件：
Download_Hypernetworks=False #@param {type:"boolean"}
#@markdown 使用NovelAi参数
Use_NovelAi_Config=True #@param {type:"boolean"}

%cd $sd_dir
!git clone https://github.com/AUTOMATIC1111/stable-diffusion-webui

!mkdir -p $webui_dir/models/Stable-diffusion $webui_dir/models/hypernetworks $webui_dir/models/deepbooru
%cd $webui_dir/models/Stable-diffusion/
!echo downloading model...
with capture.capture_output() as cap:
  !apt install -y -qq aria2
  !touch /content/empty && mv -f /content/empty ./animefull-final-pruned.ckpt && rm ./animefull-final-pruned.ckpt
  !touch /content/empty && mv -f /content/empty ./animefull-latest.ckpt rm ./animefull-latest.ckpt
  if Use_Latest_Model:
    !aria2c --summary-interval=10 -x 3 --allow-overwrite=true -Z \
      https://pub-2fdef7a2969f43289c42ac5ae3412fd4.r2.dev/animefull-latest.ckpt
    !wget -O $webui_dir/config.yaml https://pub-2fdef7a2969f43289c42ac5ae3412fd4.r2.dev/config-latest.yaml
  else:
    !aria2c --summary-interval=10 -x 3 --allow-overwrite=true -Z \
      https://pub-bbbe8fa22afd4295a465dbd52375fc55.r2.dev/animefinal-pruned.ckpt
    !wget -O $webui_dir/config.yaml https://pub-bbbe8fa22afd4295a465dbd52375fc55.r2.dev/config-pruned.yaml

if Use_NovelAi_Config:
  !wget -O $webui_dir/config.json https://raw.githubusercontent.com/PtO2/stablediffusion_webui_configs/main/config.json
  !wget -O $webui_dir/ui-config.json https://raw.githubusercontent.com/PtO2/stablediffusion_webui_configs/main/ui-config.json

# Download deepdanbooru dataset
!wget -O /content/deepbooru.zip https://github.com/KichangKim/DeepDanbooru/releases/download/v3-20211112-sgd-e28/deepdanbooru-v3-20211112-sgd-e28.zip
!unzip -o -d $webui_dir/models/deepbooru /content/deepbooru.zip
%rm -f /content/deepbooru.zip

# Install VAE Weights (optional)
!wget -O $sd_dir/animevae.pt https://cloudflare-ipfs.com/ipfs/bafybeiccldswdd3wvg57jhclcq53lvsc6gizasiblwayvhlv6eq4wow7wu/animevae.pt

%cd /content
# Install hypernetwork （optional)
if Download_Hypernetworks:
  !curl -L https://cloudflare-ipfs.com/ipfs/bafybeiduanx2b3mcvxlwr66igcwnpfmk3nc3qgxlpwh6oq6m6pxii3f77e/_modules.tar | tar x -C $webui_dir/models/hypernetworks

# Install embeddings (optional)
# !wget https://cloudflare-ipfs.com/ipfs/bafybeie3hdjchxs5tz4n75bos53nhcklslguxchdurc2ynrzcfv2kwyklu/embeddings.tar | tar x -C /content/stable-diffusion-webui/embeddings

!echo done

In [ ]:
#@title /4、更新webui代码（可选）
%cd $webui_dir

#@markdown 勾选以回退到特定版本
Rollback_Commit=False #@param {type:"boolean"}
Commit_SHA='df0a1f83815c771246a7b1bca85d63feaefad8d1' #@param {type:"string"}

%rm -f $webui_dir/.git/index.lock
!git config core.filemode false

if Rollback_Commit:
  !git reset --hard $Commit_SHA
else:
  !git reset --hard master
  !git pull
#@markdown - https://github.com/AUTOMATIC1111/stable-diffusion-webui/commits/master

In [ ]:
#@title 运行stable_diffusion_webui
from IPython.utils import capture
from subprocess import getoutput
import os,sys,time

#@markdown 启动成功后会有一个以gradio.app或loca.lt结尾的链接，点开即可访问程序

#@markdown 加载VAE文件：
Use_VAE=True #@param {type:"boolean"}

#@markdown 通过localtunnel内网穿透服务访问：
Use_Localtunnel_Share=True #@param {type:"boolean"}

#@markdown 启用xformers优化显存性能（T4机型已兼容，<font color="#dd0000">其他机型若无法出图请关闭</font>）
Enable_Xformers=True #@param {type:"boolean"}

#@markdown 加载tag自动补全插件
Load_Tagcomplete=False #@param {type:"boolean"}

#@markdown 自定义启动参数
User_Cmd_Args='--no-half-vae ' #@param {type:"string"}

!mkdir -p $webui_dir/cache/huggingface $webui_dir/cache/torch
%rm -f /root/.cache/huggingface
%rm -f /root/.cache/torch
!ln -s $webui_dir/cache/huggingface /root/.cache/
!ln -s $webui_dir/cache/torch /root/.cache/

if os.path.exists(webui_dir+'/repositories'):
  %rm -rf $webui_dir/repositories
!mkdir -p /content/repositories
%rm -f $webui_dir/repositories
!ln -s /content/repositories $webui_dir

%rm -f $webui_dir/javascript/tagAutocomplete.js
%rm -f $webui_dir/scripts/tag_autocomplete_helper.py
if Load_Tagcomplete:
  if not os.path.exists(webui_dir+'/extensions/tagcomplete') or not os.path.exists('/content/url.txt'):
    with capture.capture_output() as cap:
      !mkdir -p $webui_dir/extensions/
      if not os.path.exists(webui_dir+'/extensions/tagcomplete'):
        !git clone https://github.com/DominikDoom/a1111-sd-webui-tagcomplete.git $webui_dir/extensions/tagcomplete
      else:
        %cd $webui_dir/extensions/tagcomplete
        !git reset --hard HEAD
        !git pull
      if int(sys.version.split('.')[1])<9:
        !pip install -q pathlib3x
        !sed -i 's/from pathlib /from pathlib3x /' $webui_dir/extensions/tagcomplete/scripts/tag_autocomplete_helper.py
else:
  %rm -rf $webui_dir/extensions/tagcomplete

#grdv=getoutput("grep gradio "+webui_dir+"/requirements_versions.txt")
!sed -i 's/gradio.*/gradio==3.8/' $webui_dir/requirements.txt
!sed -i 's/gradio.*/gradio==3.8/' $webui_dir/requirements_versions.txt
!pip install -q gradio==3.8

extra_cmd=''
grdir=getoutput('pip show gradio | grep -o "/.*"')+'/gradio'
if Use_Localtunnel_Share:
  %cd /content/
  !nohup lt --port 7860 > url.txt 2>&1 &
  time.sleep(2)
  srvurl=getoutput("grep -o 'https[^ ]*' /content/url.txt")
  !sed -i 's@            self.server_name = .*@            self.server_name = "{srvurl[8:]}"@' $grdir/blocks.py
  !sed -i 's@            self.server_port = .*@            self.server_port = 443@' $grdir/blocks.py
  !sed -i 's@            self.protocol = .*@            self.protocol = "https"@' $grdir/blocks.py
  !sed -i 's@    "PUBLIC_SHARE_TRUE".*@    "PUBLIC_SHARE_TRUE": "[32mConnected",@' $grdir/strings.py
else:
  extra_cmd=extra_cmd+' --share'
  !sed -i 's@            self.server_name = .*@            self.server_name = server_name@' $grdir/blocks.py
  !sed -i 's@            self.server_port = .*@            self.server_port = server_port@' $grdir/blocks.py
  !sed -i 's@            self.protocol = .*@            self.protocol = "https" if self.local_url.startswith("https") else "http"@' $grdir/blocks.py

%cd $webui_dir
# hacks to load model in colab
#!git checkout -- ./modules/shared.py
!sed -i 's@weight_load_location =.*@weight_load_location = "cuda"@' $webui_dir/modules/shared.py

# launch
if Enable_Xformers:
  extra_cmd=extra_cmd+" --xformers"
if Use_VAE:
  extra_cmd=extra_cmd+" --vae-path "+sd_dir+"/animevae.pt"
extra_cmd=extra_cmd+' '+User_Cmd_Args
!COMMANDLINE_ARGS="--deepdanbooru --disable-safe-unpickle --gradio-debug --config config.yaml $extra_cmd" REQS_FILE="requirements.txt" python launch.py